# Merge Rhinluch files

In [86]:
# coding: utf-8
import glob
import os
import pandas as pd
import numpy as np
import shutil
from pathlib import Path
import datetime as dt
import sys

In [87]:
sddir = "sd"
remotedir = "remote"
trgdir = "merged"
tmpfile = "tmpfile.txt"
tmpfile2 = "tmpfile2.txt"
ids = ["rhi2"]

In [88]:
crns = {
     "rhi2": {"remotepattern": "lab-c-bf3-a-3_Data_*.rl2_*.txt",
         "sdpattern": "na"}
}

setups = [
          [dt.datetime(2021,3,23,12,0,0), dt.datetime(2021,5,11,10,0,0), ["rec_id", "datetime", "press4", "press1", "temp1", "relhum1", "volt", "tempCS", "relhumCS", "counts1", "nsecs1", "counts2", "nsecs2", "tempN1", "relhumN1", "tempN2", "relhumN2"]],
          [dt.datetime(2021,5,12,10,0,0), dt.datetime(2021,9,1,12,0,0), ["rec_id", "datetime", "press4", "press1", "temp1", "relhum1", "volt", "tempCS", "relhumCS", "counts1", "nsecs1", "counts2", "nsecs2", "tempN1", "relhumN1", "tempN2", "relhumN2", "rain", "sdiAdr", "sdi1_1", "sdi1_2", "sdi1_3", "sdi1_4"]]
          ]

In [89]:
for i, id in enumerate(ids):
    print("-------------")
    print("Processing %s" % id)

    try:
        os.remove(tmpfile)
        os.remove(tmpfile2)
    except:
        pass

    # REMOTE FILES
    print("Remote: ", end="")
    searchdir = os.path.join(remotedir,"%s" % id, crns[id]["remotepattern"])
    remotefiles = glob.glob(searchdir, recursive=True)
    print("found %d files" % len(remotefiles))

    allcolnames = []
    for setup in setups:
        for item in setup[2]:
            allcolnames.append(item)
    allcolnames = np.unique(allcolnames)
    
    
    # Create empty dataframe with allcolnames
    mycontainer = pd.DataFrame(columns=allcolnames)

    for name in remotefiles:
        # Find datetime
        fname = os.path.basename(name)
        fname2 = fname.split(".")[0].split("_")[2]
        ftime = dt.datetime.strptime(fname2, "%y%m%d%H%M")
#        if ftime < dt.datetime(2020,7,12):
#            continue
        foundsetup = False
        for setup in setups:
            if (ftime >= setup[0]) and (ftime < setup[1]):
                colnames = setup[2]
                foundsetup = True
                break
        if not foundsetup:
            print(fname)
            sys.exit("Could not match setup.")        
        
        # edit file into shape
        fin = open(name, "r")
        body = fin.read()
        # replace comment character
        body = body.replace("//", "#")
        # replace zombie line endings
        body = body.replace(",\r\n", "\r\n")
        # comment out these lines
        body = body.replace("CRS#1:", "#CRS#1")
        body = body.replace("CRS#2:", "#CRS#2")
        myfile = open(tmpfile, 'w')
        myfile.write(body)
        myfile.close()

            
        # MERGE
        df = pd.read_csv(tmpfile, sep=",", comment="#", header=None, error_bad_lines=False, warn_bad_lines=True)
        try:
            df.columns = colnames#crns[id]["colnames"]
        except ValueError:
            print("")
            sys.exit()
    #     if "colnames2" in crns[id].keys():
    #         try:
    #             df2 = pd.read_csv(tmpfile2, sep=",", comment="#", header=None,
    #                              error_bad_lines=False, warn_bad_lines=True)
    #             df2.columns = crns[id]["colnames2"]
    #             df = df2.append(df, sort=False)
    #         except:
    #             print("Problem in reading or appending data with diffferent column scenario")
    #             raise
        df.datetime = pd.to_datetime(df.datetime, format="%Y/%m/%d %H:%M:%S")
        #df = df.set_index("datetime")
        #df.insert(loc=1, column="datetime", value=df.index)
        #df = df.sort_index()
        #df = df[df.index >= "2019-07-25"]
        #dupl = df.index.duplicated(keep='first')
        #if np.any(dupl):
        #    print("Contains %d duplicates" % len(np.where(dupl)[0]))
        #    df = df[~dupl]
#       #sys.exit("Testend")
        mycontainer = pd.concat([mycontainer, df], sort=False)
    mycontainer = mycontainer.set_index("datetime")
    fpath = os.path.join(trgdir, "%s_CRNS.txt" % id )
    mycontainer.to_csv(fpath, sep="\t", index=True, date_format="%Y-%m-%d %H:%M:%S")
    print("Done.")


-------------
Processing rhi2
Remote: found 337 files
Done.


In [90]:
mycontainer

,counts1,counts2,nsecs1,nsecs2,press1,press4,rain,rec_id,relhum1,relhumCS,...,sdi1_1,sdi1_2,sdi1_3,sdi1_4,sdiAdr,temp1,tempCS,tempN1,tempN2,volt
datetime,,,,,,,,,,,,,,,,,,,,,
2021-03-23 13:32:25,12,11,31,32,1018.4,1020.31,NaN,1,78.2,82.1,...,NaN,NaN,NaN,NaN,NaN,7.0,6.2,7.4,8.5,14.617
2021-03-23 13:51:00,320,354,878,879,1018.4,1019.94,NaN,3,72.8,82.4,...,NaN,NaN,NaN,NaN,NaN,8.1,6.2,10.9,10.2,14.615
2021-03-23 14:11:00,455,477,1200,1200,1018.4,1019.73,NaN,4,69.2,81.8,...,NaN,NaN,NaN,NaN,NaN,8.6,6.1,11.5,10.5,14.581
2021-03-23 14:31:00,460,515,1200,1200,1018.3,1019.77,NaN,5,58.7,83.6,...,NaN,NaN,NaN,NaN,NaN,8.7,6.0,11.7,10.6,13.377
2021-03-23 14:51:00,446,428,1200,1200,1018.1,1019.68,NaN,6,51.1,84.0,...,NaN,NaN,NaN,NaN,NaN,8.4,6.0,11.7,10.5,14.611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-15 09:04:00,467,436,1200,1200,1014.8,1016.01,0,2446,31.7,75.1,...,19.608,23.492,30.172,41.755,sdi1,23.8,19.3,26.0,24.1,13.729
2021-06-15 09:24:00,463,490,1200,1200,1014.9,1016.21,0,2447,31.8,74.1,...,19.601,23.486,30.159,41.722,sdi1,24.0,19.6,26.0,24.1,13.690
2021-06-15 09:44:00,471,472,1200,1200,1015.0,1016.18,0,2448,31.6,73.1,...,19.582,23.479,30.134,41.734,sdi1,24.7,19.7,26.1,24.3,13.680


In [77]:
for setup in setups:
    if (ftime >= setup[0]) and (ftime < setup[1]):
        print(ftime)
        colnames = setup[2]
        foundsetup = True

2021-05-12 10:25:00


In [82]:
name

'remote\\rhi2\\lab-c-bf3-a-3_Data_2105121025.rl2_000001.txt'

In [81]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,2021/03/23 13:32:25,1020.31,1018.4,7.0,78.2,14.617,6.2,82.1,12,31,11,32,7.4,0.0,8.5,0.3
1,3,2021/03/23 13:51:00,1019.94,1018.4,8.1,72.8,14.615,6.2,82.4,320,878,354,879,10.9,0.0,10.2,0.3
2,4,2021/03/23 14:11:00,1019.73,1018.4,8.6,69.2,14.581,6.1,81.8,455,1200,477,1200,11.5,0.0,10.5,0.3
3,5,2021/03/23 14:31:00,1019.77,1018.3,8.7,58.7,13.377,6.0,83.6,460,1200,515,1200,11.7,0.0,10.6,0.3
4,6,2021/03/23 14:51:00,1019.68,1018.1,8.4,51.1,14.611,6.0,84.0,446,1200,428,1200,11.7,0.0,10.5,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3578,3580,2021/05/12 06:11:00,1004.07,1002.0,13.6,55.3,13.914,11.9,92.0,515,1200,505,1200,17.2,0.0,15.6,1.7
3579,3581,2021/05/12 06:31:00,1003.91,1001.9,13.9,56.0,13.601,12.0,90.8,480,1200,540,1200,17.3,0.0,15.7,1.7
3580,3582,2021/05/12 06:51:00,1004.06,1002.1,14.0,56.1,13.698,12.0,91.2,518,1200,528,1200,17.5,0.0,15.9,1.7
3581,3583,2021/05/12 07:11:00,1004.27,1002.2,14.0,56.3,13.541,12.0,91.8,508,1200,482,1200,17.6,0.0,16.0,1.7


In [53]:
 pd.read_csv(name, sep=",", skiprows=2, header=None, error_bad_lines=False, warn_bad_lines=True)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,2021/03/23 13:32:25,1020.31,1018.4,7.0,78.2,14.617,6.2,82.1,12,31,11,32,7.4,0.0,8.5,0.3
